# Profile Domain Code

Below is the profile domain code I've written in Jupyter notebook form. It's useful to play around/test with, but I wouldn't actually run the code from here because (as I understand it) you'd have to keep the notebook running the entire time the code runs, which is a loooooooong time.

Anyway, I won't go into much conceptual detail here, because hopefully we'll cover that when we meet, but rather will try to explain the code as much as I can first. Note that this was written using python3 (since it uses PyMC3, which needs python3). I think bowser defaults to python2, so you might need to ask Nate to set up python3 for you.

Anywhoo, to run this, you'll need 3 things:

1. A par file. Easy enough.
2. A "metafile". This is just a text file with each line being the name of a data file you'd like to include in the timing solution. If you want an example, there's one here: `/hyrule/data/users/pgentile/prof_domain/Profile_Domain_Timing/datafiles.txt`
3. A "model file". This is a file that contains a list of the parameters of the gaussians that have been fit to the profile. You'll have to make this yourself. To do that, take the metafile from step 2 and run the following command:

    `python2 /home/pgentile/.local/lib/python2.7/site-packages/ppgauss.py -M metafile_you_made.txt`
    
    That'll bring up a display with a pulse profile, and will print out some instructions on how to fit gaussians to the profile. Follow them and get a "good enough" fit. Then press "p" to print out that fit. The text that gets spat out can be copied and pasted into a text file.
    
    Note that this code (the ppgauss one) needs to be run in python2. I know this is super annoying, and it's pretty much my fault, but it's a huge pain to fix, and, well, to be honest it was more important to work on other parts of the code first. Sorry :(.

Once you have those files, you should be set to start running the code. 

For this first block, I'm just importing the tons of modules to be used. Not much insight to be gleaned here, but you need to have installed them all. If you don't have something (like let's say for example you probably don't have pymc3), you can istall it by doing a `pip install pymc3 --user`. If something breaks, Nate should be able to help.

One thing to know is that this code assumes the data have been folded with something like `fold_psrfits`.

We talked on Friday about Lindley's first paper on this, and I said I'd send it to you. The url is here:

https://arxiv.org/pdf/1412.1427.pdf


In [1]:
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import pickle, time, pint, theano, pypulse, dill
import theano.tensor as T
import pint.models.model_builder as mb
from pint.models.parameter import floatParameter as fp
import pint.toa as toa
import astropy.units as u
from astropy import log

LOOK HERE!!!
 /home/pgentile/.theano/compiledir_Linux-2.6-el6.x86_64-x86_64-with-centos-6.8-Final-x86_64-3.6.3-64


Again, not too much to be gleaned from this next block. The first two lines tell the code how verbose to be. PINT prints errors and warnings for a ton of things through astropy and since they're not important and super annoying, I just shut them off unless it breaks the code (i.e. it's an actual `ERROR`). Theano is the opposite, where if it prints out a warning or an error, it's usually catastrophic, so I've made it very verbose. This still makes the errors it prints out almost completely useless, but it's fun to marvel at.

For the purposes of a jupyter notebook, I've also hardcoded the names of the three necessary files listed above, so they're there.

Aside from that, it sets you load a trace you've made before.

In [2]:
log.setLevel("ERROR")
theano.config.exception_verbosity='high'
load_from_trace = False 
make_init = True
modelfile = "Profile_Domain_Timing/J1234-3630_model.gmodel"
parfile = "Profile_Domain_Timing/1235-36.par"
metafile = "Profile_Domain_Timing/datafiles.txt"

This block makes a function to read in the modelfile above, which is just a bunch of gaussians. It also loads in things we don't end up using, like `CODE`and `ALPHA`, but one day we might use them, so hey, why not?

In [ ]:
def get_params(pfile):
    plines = open(pfile).readlines()
    gparams = []
    mus = []
    sds = []
    coeffs = []
    for l in plines:
        s = l.split()
        if l.startswith("CODE"):
            code = s[1]
        elif l.startswith("FREQ"):
            nu_ref = float(s[1])
        elif l.startswith("DC"):
            DC = float(s[1])
        elif l.startswith("TAU"):
            tau = float(s[1])
        elif l.startswith("ALPHA"):
            alpha = float(s[1])
        elif l.startswith("COMP"):
            mus += [float(s[1])+0.3] 
            sds += [float(s[5])/(2 * np.sqrt(2 * np.log(2)))]
            coeffs += [float(s[9])]
    model = [DC] + [tau] + gparams
    mus = np.asarray(mus)
    sds = np.asarray(sds)
    coeffs = np.asarray(coeffs)
    return code, nu_ref, alpha, DC, tau, mus, sds, coeffs

This block is a little tricky. It takes in the gaussian parameters from above and makes a function that takes in a phase and spits out the template profile intensity at that phase. It then returns that function. Note, you need to do some funny stuff to deal with components that wrap around from phase 0.9 to 0.1 (for example).

In [3]:
def psr_model(DC, mus, sds, coeffs):
    def profile(phases):
        intens = np.zeros_like(phases, dtype=np.float32)+DC
        '''intens = np.zeros_like(phases)+DC'''
        for mu, sd, coeff in zip(mus, sds, coeffs):
            first = coeff*np.exp((-((phases+1) - mu)**2)/(2*sd**2))
            second = coeff*np.exp((-((phases+1) - (mu+1))**2)/(2*sd**2))
            third = coeff*np.exp((-((phases+1) - (mu+2))**2)/(2*sd**2))
            intens += (first+second+third).astype(np.float64)
        norm_intens = (1.0/intens.max())*intens
        return norm_intens
    return profile

This block really just combines the previous two blocks so you can go straight from modelfile to profile function.

In [4]:
def get_profile_model(modelfile):
	[model_DC, model_tau, model_mus, model_sds, model_coeffs] = get_params(modelfile)[3:]
	return psr_model(model_DC, model_mus, model_sds, model_coeffs)

This next block uses PyPulse (written by our own Michael Lam) to open up an observation file (that's been folded either by the telesope or with fold_psrfits), grab a bunch of useful information like the start MJD and telescope, and grab the data. It then returns all that stuff. Tbin is the time per phase bin, and the "weights" bit just zaps all the channels the user wanted to zap.

In [5]:
def get_data(dfile):
	observatory = "ao"
	ar = pypulse.Archive(dfile)
	ar.dededisperse()
	weights = ar.getWeights()
	data = ar.getData()[weights>0,:]
	nbin = ar.getNbin()
	tbin = ar.getTbin()
	freqs = ar.freq.squeeze()[weights>0]
	mjd = ar.header["STT_IMJD"] + (ar.header["STT_SMJD"]+ar.header["STT_OFFS"])/86400.0
	data /= (np.amax(data, axis=1)[:,None])
	
	scope = ar.getTelescope()
	if scope == "GBT":
		observatory = "gb"
	elif scope == "Arecibo":
		observatory = "ao"
	
	return data, freqs, nbin, tbin, mjd, observatory


This next block just loops through all the data files in your text file full of file names and uses the `get_data` function defined above to get the data in them. It then stores all that into two big arrays and returns them.

In [8]:
def get_all_data(meta_file_name):
	dfnames = [x.replace("\n","") for x in  open(meta_file_name).readlines()]
	data_array = []
	meta_array = []
	for f in dfnames:
		data, freqs, nbin, tbin, mjd, obs = get_data(f)
		data_array.append(data)
		meta_array.append([freqs, nbin, tbin, mjd, obs])
	
	data_array = np.asarray(data_array)
	meta_array = np.asarray(meta_array)
	return data_array, meta_array

Honestly I don't remember why, but I really wanted to make the timing model have a spin period (in addition to spin frequency, if it had that instead). So I wrote this to make it so.

In [9]:
def get_period(timing_model):
	try:
		period = timing_model.P0.value
	except AttributeError:
		p = fp(name = "P0", value = 1.0 / timing_model.F0.value, units = "s")
		timing_model.add_param_from_top(p, "")
		period = timing_model.P0.value
	
	return period

This next block takes the PINT timing model pbject and makes a dictionary out of it. It's useful because

1. Dictionaries play nice with basically all python code and
2. It only keeps the "unfrozen" parameters (that is, the ones you're actually fitting for).

So with this, if you can make a python variable that you call something like `timing_model` and do `timing_model["DM"]` or `timing_model["F0"]` and it'll give you the DM or spin frequency.

In [11]:
def get_timing_model_pardict(timing_model):
	pars = timing_model.params
	pardict = {}
	for p in pars:
		par = getattr(timing_model, p)
		if not par.frozen:
			pardict[p] = par
	return pardict

This next block is pretty efficiently written, which means its a little hard to read, but lets say you want to view a pulse profile starting at a phase of 0.23 and coverting a full rotation (so, ending at a phase of 1.23), you need to make phases that go from 0.23 to 1, then wrap back around to 0, and continue to 0.23 again. This block does that.

The other important thing it does is take in a time (in the form of a TOA) and calculate the phase at that time. This part is done with the `model.phase(toas).frac.value` part.

In [1]:
def make_phases(freqs, toas, model, nbin, sec_per_bin):
	phs = np.tile(np.linspace(0,1,nbin),  (freqs.shape[0],1))
	start_phases = np.asarray([x%1 for x in model.phase(toas).frac.value])
	phases = start_phases[:,None] + phs
	return np.mod(phases,1, dtype=np.float32)

Think of a TOA as you'd see it in a `.tim` file. It's not just an MJD, right? It also includes the frequency and observatory. All this is doing is taking the MJD of the start of an observation the observatory used to take the data, and the frequency of that data (as found in the `metafile` list that's passed to it), and making that into a PINT `TOA` object.

In [ ]:
def make_TOAs(metadata):
	TOA_list = []
	for i in range(metadata.shape[1]):
		freqs, nbin, tbin, mjd, observatory = metadata[i]
		new_toas = [toa.TOA((np.modf(mjd)[1], np.modf(mjd)[0]), obs = observatory, freq = freq) for freq in freqs]
		TOA_list.append(toa.get_TOAs_list(new_toas))
	
	return TOA_list

When you vary the parameters in the par file over and over again to try to find which parameters best describe the data, samplers will often spend some time trying to figure out the best way to do that before actually going at it. This is called "initializing the sampler", and it can really take some time. This code lets you save that initialization so if you want to run the code over again (to take more samples), you can just go straight to taking the samples while retaining the benefit of the initialization you did before.

Note that I don't think I actually use this, but it's definitely something that's nice to have.

In [2]:
def save_init(init_approx, fname="dm_init.pkl"):
	bij = init_approx.approx.groups[0].bij
	save_param = {param.name: bij.rmap(param.eval()) for param in init_approx.approx.params}
	with open(fname, "wb") as initfile:
		pickle.dump(save_param, initfile)
	
	print("saved")

If you have a way to save the initialization from before, you darned well better have a way to load an initialization you've already saved! This does that.

In [ ]:
def load_init(init_fname, from_dict=False):
	if not from_dict:
		with open(init_fname, "rb") as initfile:
			save_param = pickle.load(initfile, encoding="latin1")
	else:
		save_param = {'mu': {'DM': np.array(14.326125), 'phase_offset_interval__': np.array(0.70)}, 'rho': {'DM': np.array(0.5), 'phase_offset_interval__': np.array(0.2)}}
		
	init_approx = pm.ADVI()
	bij = init_approx.approx.groups[0].bij
	for i,p in enumerate(init_approx.approx.params):
		init_approx.approx.params[i].set_value(bij.map(save_param[p.name]))
	
	return init_approx

This is the actual log likelihood. All that's going on here is you're taking the pulse intensity as predicted by your template at a given phase, and comparing that to the data at that phase in the same way as what we talked about when we met.

In [ ]:
def loglike(t_mod, profile_model, toas, metadata, data, noise, offset):
	freqs, nbin, sec_per_bin, tr1, tr2 = metadata
	phases = (make_phases(freqs, toas, t_mod, nbin, sec_per_bin)+offset)%1
	lprobs = -0.5*np.log(2*np.pi*noise**2) + np.sum(-(profile_model(phases) - data)**2)/ (2*np.pi*noise**2)
	return lprobs

The most powerful sampling algorithms (like the one we'll use, called NUTS) require the gradient of the log likelihood WRT (with respect to) the parameters you're varying with the sampler. This lets the sampler answer the question, "If I wanted to increase the log likelihood, do I need to increase or decrease RA? F0? DM?" That way it can intelligently vary the parameters instead of just jumping around randomly and wasting a ton of time.

Unfortunately that means you'll need to actually calculate the gradient. There's nothing clever going on here, I just wrote down the log likelihood (literally, on a blackboard) and took the derivative of it WRT the timing parameters we'll be sampling. When you do that you'll get some things that need explaining:

First, you'll eventually find you'll need to calculate the change in phase WRT the change in the parameters. So basically if you give me some MJD and some timing model, I can tell you the phase at that MJD, but I also want to know how much that phase would change if I increased the DM by 1. Or the binary period. Or whatever. This "change of phase WRT timing model parameters" is calculated automatically by PINT, which calls it the "design matrix". So all the `t_mod.designmatrix(TOAs)` part is doind is calculating the change in phase WRT the timing parameters

Second, you'll eventually find you need to know how much the pulse intensity changes at a given phase WRT timing parameters. It's subtle, but this is different from the last thing we talked about. To see why, think about a pulse profile that just has one really narrow component. Let's say that component spans a phase of 0.5 to 0.10, and outside of that, there's no pulse intensity. Now think about what the pulse intensity looks like at a phase of 0.8. At first, there's no intensity. Now, let's rotate the pulse profile by a phase of 0.5 (a big rotation). So now the component spans phases of 0.55 to 0.6. How has the intensity at a phase of 0.8 changed? *It hasn't.* So for this pulsar, even big changes in phase don't necessarily mean a big change in intensity. 

I've called this "pulse intensity change at a given phase WRT timing parameters" part `P_grad`, which stands for "Profile Gradient". I calculate it in a block I'll put right below this one.

Other than that, the only things I really do are to make sure the gradient of the log likelihood is in the right shape and return it.

In [ ]:
def gradient(t_mod, noise, offset, profile_model, metadata, P_gradient, data, TOAs):
	freqs, nbin, sec_per_bin, tr1, tr2 = metadata
	phases = (make_phases(freqs, TOAs, t_mod, nbin, sec_per_bin)+offset)%1
	diffs = profile_model(phases) - data
	P_grad = P_gradient(phases)
	d_phase_d_psr_pars = -t_mod.designmatrix(TOAs)[0][:,1:]
	ones_column = np.ones((d_phase_d_psr_pars.shape[0], d_phase_d_psr_pars.shape[1]))#[:,None]
	d_phase_d_pars = np.append(d_phase_d_psr_pars, 1*ones_column, axis=1)
	data_part = np.sum((diffs/(noise**2))*P_grad, axis=1)
	grad = np.sum(data_part[:,None] *d_phase_d_pars, axis=0)
	return grad

Here's where I actualy make `P_grad`:

In [3]:
def P_gradient(DC, mus, sds, coeffs):
	def profile(phases):
		intens = np.zeros_like(phases, dtype=np.float32)+DC
		'''intens = np.zeros_like(phases)+DC'''
		for mu, sd, coeff in zip(mus, sds, coeffs):
			first = (coeff*np.exp((-((phases+1) - mu)**2)/(2*sd**2)))*((mu - (phases+1))/(sd**2))
			second = (coeff*np.exp((-((phases+1) - (mu+1))**2)/(2*sd**2)))*(((mu+1) - (phases+1))/(sd**2))
			third = (coeff*np.exp((-((phases+1) - (mu+2))**2)/(2*sd**2)))*(((mu+2) - (phases+1))/(sd**2))
			intens += (first+second+third).astype(np.float64)
		norm_intens = (1.0/intens.max())*intens
		return norm_intens
	return profile

def get_P_gradient(modelfile):
	[model_DC, model_tau, model_mus, model_sds, model_coeffs] = get_params(modelfile)[3:]
	return P_gradient(model_DC, model_mus, model_sds, model_coeffs)

Whelp, here we go. This is where the train goes off the rails. The big challenge with this code is that the type of things we're doing are totally unlike what most people do. We're not saying, "well I've gathered test scores for an entire 200 person class. The distribution looks like a gaussian, so fit a gaussian to it and tell me what the mean and standard deviation is." This would be easy: gaussians are coded into essentially every stats module in existence.

No, we're saying, "ok, we have a pulse profile, which could be any crazy shape. And we want to see how it's time of arrival is changed by some parameters that are included in some text file somewhere which could include lots of crazy things, and sample those parameters in an intelligent way. Also, we need to correct for the position of the Earth, Solar system shapiro delay, and barycenter the times of arrival." There is no module which has this functionality, so we need to make it ourselves. We have PINT, which can do a lot of the pulsar-y things above (correct for shapiro delay and barycenter the data), and we have PyMC3, which can sample parameters in an intelligent way. But these two things cannot talk to each other. So we need to make them talk to each other.

This means a lot of what follows will inherit some of the oddities of the way PyMC3 works. For example, math in PyMC3 is done with a module called `theano`, which is some strange Python and C lovechild. It's super fast, but (for example) it means if you make a class that you want theano to be able to use, it has to have a `perform` function inside it. Also, that fuction needs to take an argument called "node". Why? Haven't the foggiest! It's just the way it needs to be done. Sorry about that, but it's unavoidable.

Anyway, I'll try my best to explain what all the functions below do the best I can. I'm sure you'll be left wanting, but spare a moment for the guy who actually had to write this code.

`__init__` - You'll see a lot of familiar faces here. This will take in the filenames you'll work with and read in the data and timing model, make the profile model, make the pardict, and set up the gradient function.


`make_period` - This isn't really used anywhere, but it does the same thing as described above.

`get_timing_model_pardict` - This does the same thing as described above.

`make_pymc3_model` - Isn't used. I should just delete it.

`update_timing_model` - When PyMC3 picks new values for the timing model parameters, we want to tell PINT about it and have PINT make a new timing model with those parameters. Lucky, in terms of syntax, this is easy (thanks, PINT!).

`fit_model` - Isn't used. I should just delete it.

`perform` - Ok, so this is the function that actually calculates the log likelihood. This is the function that PyMC3 calls when it chooses new parameter values (and passes them using the name `inputs`). So when it does that, we want to actually get those new parameter values, update our timing model, calculate the log likelihood, and return it. Normally, when you want to return something from a Python function, you use `return`, but with theano (and therefore PyMC3), you need to store it as `outputs`. So the `outputs[0][0] = np.array(total_logl)` part is essentially the theano equivalent of `return total_logl`.

`old_grad` - I used this for a bit for debugging purposes, but it isn't currently used. I should just delete it.

`grad` - As explained above, we need to calculate the gradient of the log likelihood WRT the timing parameters every time new parameters are chosen. This does that (in a necessarily roundabout way). Thankfully (and aggrivatingly), you can actually use `return` here.



In [ ]:
class LogLikeWithGrad(T.Op):
	itypes = [T.dvector]
	otypes = [T.dscalar]
	
	def __init__(self, loglike, filenames):
		#print("Initializing LLWG")
		self.loglike = loglike
		self.dfile, self.parfile, self.modelfile = filenames
		self.ofile = open("dm_samples.txt", "w")
		#self.data, self.freqs, self.nbin, self.sec_per_bin, self.mjdref = get_data(dfile)
		self.data, self.metadata = get_all_data(self.dfile)
		self.profile_model = get_profile_model(modelfile)
		self.P_gradient = get_P_gradient(modelfile)
		self.t_mod = mb.get_model(parfile)
		self.period = get_period(self.t_mod)
		self.init_pepoch = self.t_mod.POSEPOCH.value
		self.init_DM = self.t_mod.DM.value
		self.pardict = self.get_timing_model_pardict()
		self.TOAs = make_TOAs(self.metadata)
		self.logpgrad = LogLikeGrad(self.data, self.metadata, self.TOAs, self.profile_model, self.P_gradient, self.pardict, self.t_mod)
		#print("Initialized")
	
	def make_period(self):
		try:
			period = self.t_mod.P0.value
		except AttributeError:
			p = fp(name = "P0", value = 1.0 / self.t_mod.F0.value, units = "s")
			self.t_mod.add_param_from_top(p, "")
	
	def get_timing_model_pardict(self):
		pars = self.t_mod.params
		pardict = {}
		for p in pars:
			par = getattr(self.t_mod, p)
			if not par.frozen:
				pardict[p] = par
		return pardict
	
	def make_pymc3_model(self):
		basic_model = pm.Model()
		basic_model_pars = []
		with basic_model:
			for p in self.pardict.keys():
				par = self.pardict[p]
				print(type(par.value), type(par.uncertainty_value))
				vars()[p] = pm.Normal(par.name, mu=float(par.value), sd=float(par.uncertainty_value))
				basic_model_pars.append(vars()[p])
			noise = pm.Bound(pm.Normal, lower=0)("noise", mu=0.66, sd = 0.5)
			basic_model_pars.append(noise)
			theta = T.as_tensor_variable(basic_model_pars)
			Y_obs = pm.DensityDist("Y_obs", lambda v: self.perform(v), observed={'v': theta})
		
		return basic_model
	
	def update_timing_model(self, theta):
		for key, theta_par in zip(self.pardict.keys(), theta):
			#print(dir(theta_par))
			#print(type(theta_par))
			self.pardict[key].value = theta_par
	
	def fit_model(self, nits=5000, ntune = 3000, init = "advi"):
		with self.pymc3_model:
			self.trace = pm.sample(5000, tune=3000, chains=4, init = "advi")
	
	def perform(self, node, inputs, outputs):
		#print("Performing!")
		#print(inputs[0])
		theta, = inputs
		#print(theta[:-2])
		#print(theta[-1])
		#print(theta[-2])
		#self.ofile.write(" ".join([str(x) for x in theta]) + "\n")
		self.update_timing_model(theta[:-1])
		#logl = self.loglike(self.t_mod, self.profile_model, self.freqs, self.data, theta[-2], theta[-1])
		#print(self.sec_per_bin)
		total_logl = 0
		for d, m, ts in zip(self.data, self.metadata, self.TOAs):
			logl = self.loglike(self.t_mod, self.profile_model, ts, m, d, 4.0, theta[-1])
			total_logl = total_logl + logl
		print("\nperform: " + " ".join([str(x) for x in theta] + [str(logl)]))
		
		outputs[0][0] = np.array(total_logl)
	
	def old_grad(self, inputs, g):
		'''
		the method that calculates the gradients - it actually returns the
        vector-Jacobian product - g[0] is a vector of parameter values
		'''
		#print("Old Grad-ing!")
		theta = inputs[0].eval()
		self.update_timing_model(theta[:-1])
		grads = gradient(self.t_mod, theta[-1], self.profile_model, self.data, self.freqs,
						self.TOAs)
		return [g[0]*grads]
	
	def grad(self, inputs, g):
		theta, = inputs
		grads = self.logpgrad(theta)
		return [g[0]*grads]
	


Because nothing in this world is easy, PyMC3 wants you to make the gradient into it's own class, and because it's a class used by PyMC3, it needs to have a `perform` function, which mysteriously takes in a "node" variable. As before, the results aren't returned, but stored as the `outputs` variable. Mercifully, there are only 3 functions for this class:

`__init__` - As before, it just sets up some initial variables. Check to see how it's called from `LogLikeWithGrad` if you need more clarity.

`update_timing_model` - Because this is a different class than `LogLikeWithGrad`, updating the timing model in `LogLikeWithGrad` won't update the timing model here, so we need to be able to update the timing model here, too.

`perform` - Like with `LogLikeWithGrad`, this is the function that actually does the thing. This takes in timing model parameters, calculates the gradient of the log likelihood with respect to them, then stores the result as `outputs`.

In [ ]:
class LogLikeGrad(T.Op):
	'''
	This Op will be called with a vector of values and also return a vector of
	values - the gradients in each dimension.
	'''
	itypes = [T.dvector]
	otypes = [T.dvector]
    
	def __init__(self, data, metadata, TOAs, profile_model, P_gradient, pardict, t_mod): #Plus anything else needed to calculate the gradient that doesn't change with new pars
		self.data = data
		self.metadata = metadata
		self.TOAs = TOAs
		self.profile_model = profile_model
		self.pardict = pardict
		self.t_mod = t_mod
		self.P_gradient = P_gradient
	
	def update_timing_model(self, theta):
		for key, theta_par in zip(self.pardict.keys(), theta):
			self.pardict[key].value = theta_par
	
	def perform(self, node, inputs, outputs): 
		'''
		This actually calculates the gradient. Need t_mod and noise. I'm assuming
		inputs is whatever we call the functions with in LogLikeWithGrad, rather
		than some other thing PYMC3 just shoves in there. So we can do:
		'''
		#print(type(inputs[0]))
		theta, = inputs
		self.update_timing_model(theta[:-1])
		#print("Theta shape", theta.shape)
		#print(theta[:-1])
		#noise = theta[-2]
		noise = 4.0
		offset = theta[-1]
		total_grads = 0
		for d, m, ts in zip(self.data, self.metadata, self.TOAs):
			grads = gradient(self.t_mod, noise, offset, self.profile_model, m, self.P_gradient, d, ts)
			total_grads = total_grads + grads
		
		print(total_grads.shape, theta.shape)
		outputs[0][0] = total_grads

Here, we're actually doing stuff. We're using the `LogLikeWithGrad` class we made before to set up a likelihood function, then we're defining prior (i.e. initial) distributions of the timing model parameters. The Y_obs is the actualy likelihood being implemented. Then we actually sample! That's the `pm.sample` part. Check out the documentation of that function to see more details.

In [ ]:
llwg = LogLikeWithGrad(loglike, [metafile, parfile, modelfile])
basic_model = pm.Model()
basic_model_pars = []
with basic_model:
	for p in llwg.pardict.keys():
		par = llwg.pardict[p]
		vars()[p] = pm.Normal(par.name, mu=float(par.value), sd=float(10000*par.uncertainty_value), testval = float(par.value))
		basic_model_pars.append(vars()[p])
	phase_offset = pm.Bound(pm.Normal, lower=0, upper=1)("phase_offset", mu=0.46, sd = 0.2, testval=0.46)
	basic_model_pars.append(phase_offset)
	theta = T.as_tensor_variable(basic_model_pars)
	Y_obs = pm.DensityDist("Y_obs", lambda v: llwg(v), observed={'v': theta})
	stime = time.time()
	print("Began MC stuff at", time.ctime())
	if not load_from_trace:
		trace = pm.sample(draws = 2000, tune = 1000, chains=1, init = "advi")
		pm.save_trace(trace)
	else:
		trace = pm.load_trace(tracefile)



That's really it! Here, we're just plotting and saving a diagnostic plot, then exiting. 

In [ ]:
pm.traceplot(trace)
plt.savefig(dfile + ".ps")

plt.show()

print("MCMC stuff finished in", (time.time()-stime)/60.0, "minutes.")

exit(0)


Note that after the above code, I have more stuff, but I don't use it. It's just vestigial code from me trying stuff. You can just delete it if you want and it won't change anything, so I'm not going to bother explaining any of it here.